In [4]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

pd.set_option('display.max_columns', 500)

In [5]:
train = pd.read_csv('train_v2_flat.csv',dtype={'date': str, 'fullVisitorId': str, 'sessionId':str, 'visitId': np.int64})

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [6]:
test = pd.read_csv('test_v2_flat.csv',dtype={'date': str, 'fullVisitorId': str, 'sessionId':str, 'visitId': np.int64})

In [7]:
#lets look at columns
train.head()

,channelGrouping,date,fullVisitorId,visitId,visitNumber,visitStartTime,device_browser,device_deviceCategory,device_isMobile,device_operatingSystem,geoNetwork_city,geoNetwork_continent,geoNetwork_country,geoNetwork_metro,geoNetwork_networkDomain,geoNetwork_region,geoNetwork_subContinent,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adContent,trafficSource_adwordsClickInfo.adNetworkType,trafficSource_adwordsClickInfo.gclId,trafficSource_adwordsClickInfo.isVideoAd,trafficSource_adwordsClickInfo.page,trafficSource_adwordsClickInfo.slot,trafficSource_campaign,trafficSource_campaignCode,trafficSource_isTrueDirect,trafficSource_keyword,trafficSource_medium,trafficSource_referralPath,trafficSource_source
0,Organic Search,20171016,3162355547410993243,1508198450,1,1508198450,Firefox,desktop,False,Windows,not available in demo dataset,Europe,Germany,not available in demo dataset,(not set),not available in demo dataset,Western Europe,1.0,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,water bottle,organic,NaN,google
1,Referral,20171016,8934116514970143966,1508176307,6,1508176307,Chrome,desktop,False,Chrome OS,Cupertino,Americas,United States,San Francisco-Oakland-San Jose CA,(not set),California,Northern America,NaN,2,NaN,2.0,2.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,NaN,referral,/a/google.com/transportation/mtv-services/bike...,sites.google.com
2,Direct,20171016,7992466427990357681,1508201613,1,1508201613,Chrome,mobile,True,Android,not available in demo dataset,Americas,United States,not available in demo dataset,windjammercable.net,not available in demo dataset,Northern America,NaN,2,1.0,2.0,1.0,38.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,True,NaN,(none),NaN,(direct)
3,Organic Search,20171016,9075655783635761930,1508169851,1,1508169851,Chrome,desktop,False,Windows,not available in demo dataset,Asia,Turkey,not available in demo dataset,unknown.unknown,not available in demo dataset,Western Asia,NaN,2,1.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google
4,Organic Search,20171016,6960673291025684308,1508190552,1,1508190552,Chrome,desktop,False,Windows,not available in demo dataset,Americas,Mexico,not available in demo dataset,prod-infinitum.com.mx,not available in demo dataset,Central America,NaN,2,1.0,2.0,1.0,52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,(not set),NaN,NaN,(not provided),organic,NaN,google


In [8]:
#What is shape of df?
print(train.shape)
print(test.shape)

(1708337, 39)
(401589, 38)


In [9]:
#what percent of rows are nan?
revenue = train['totals_transactionRevenue']
revenue.isna().sum()/len(revenue)

0.989162559846213

In [10]:
#what percent of visitors have no revenue when visits combined?  This indicates unbalanced dataset.

concatrev = train.groupby(train['fullVisitorId']).sum()
concatrev = concatrev['totals_transactionRevenue']
mask = concatrev[concatrev > 0]
1-(len(mask))/len(concatrev)


0.9878064257816926

In [11]:
#What is breakdown of visit number?
train['visitNumber'].describe()

#mostly people visit the site once but outliers driving this.  Could we create column for "power user?"


count    1.708337e+06
mean     2.335170e+00
std      9.354034e+00
min      1.000000e+00
25%      1.000000e+00
50%      1.000000e+00
75%      1.000000e+00
max      4.570000e+02
Name: visitNumber, dtype: float64

In [12]:
#print out unique values per column, will help with variable selection

for x in train.columns:
    print(x, len(train[x].unique()))





channelGrouping 8
date 638
fullVisitorId 1323730
visitId 1665802
visitNumber 457
visitStartTime 1667423
device_browser 129
device_deviceCategory 3
device_isMobile 2
device_operatingSystem 24
geoNetwork_city 956
geoNetwork_continent 6
geoNetwork_country 228
geoNetwork_metro 123
geoNetwork_networkDomain 41982
geoNetwork_region 483
geoNetwork_subContinent 23
totals_bounces 2
totals_hits 297
totals_newVisits 2
totals_pageviews 231
totals_sessionQualityDim 101
totals_timeOnSite 4775
totals_totalTransactionRevenue 8507
totals_transactionRevenue 7252
totals_transactions 14
trafficSource_adContent 77
trafficSource_adwordsClickInfo.adNetworkType 4
trafficSource_adwordsClickInfo.gclId 59009
trafficSource_adwordsClickInfo.isVideoAd 2
trafficSource_adwordsClickInfo.page 12
trafficSource_adwordsClickInfo.slot 4
trafficSource_campaign 33
trafficSource_campaignCode 2
trafficSource_isTrueDirect 2
trafficSource_keyword 4547
trafficSource_medium 7
trafficSource_referralPath 3197
trafficSource_source 345

In [13]:
#only keeping columns that have multiple unique values

colskeep = []
for x in train.columns:
    if len(train[x].unique()) != 1:
        colskeep.append(x)

#going to remove columns that not in test set so df have same cols.  Target variable already saved.

colskeep.remove('trafficSource_campaignCode')
trainnew = train[colskeep]
testnew = test[colskeep]


trainnew.shape
        

(1708337, 38)

In [14]:
#null values in df?  #mostly not, but where there are nulls, high percentage of columns.  Will figure out how to address later.
trainnew.isna().sum()/len(trainnew)

channelGrouping                                 0.000000
date                                            0.000000
fullVisitorId                                   0.000000
visitId                                         0.000000
visitNumber                                     0.000000
visitStartTime                                  0.000000
device_browser                                  0.000000
device_deviceCategory                           0.000000
device_isMobile                                 0.000000
device_operatingSystem                          0.000000
geoNetwork_city                                 0.000000
geoNetwork_continent                            0.000000
geoNetwork_country                              0.000000
geoNetwork_metro                                0.000000
geoNetwork_networkDomain                        0.000000
geoNetwork_region                               0.000000
geoNetwork_subContinent                         0.000000
totals_bounces                 

In [15]:
train['trafficSource_adwordsClickInfo.page'].describe()

count    75274.000000
mean         1.025268
std          0.227444
min          1.000000
25%          1.000000
50%          1.000000
75%          1.000000
max         14.000000
Name: trafficSource_adwordsClickInfo.page, dtype: float64

In [16]:
# correlation of numeric variables, anything correlate with revenue?

train.corr()

#visit number, totals.hits, and totals.pageviews strongest corr with rev
#hits and pageviews strongly correlated, may want to remove one for ols

,visitId,visitNumber,visitStartTime,device_isMobile,totals_bounces,totals_hits,totals_newVisits,totals_pageviews,totals_sessionQualityDim,totals_timeOnSite,totals_totalTransactionRevenue,totals_transactionRevenue,totals_transactions,trafficSource_adwordsClickInfo.page
visitId,1.000000,0.010963,1.000000,0.126059,NaN,-0.024275,NaN,-0.027545,0.000363,-0.015798,-0.021421,-0.026284,-0.025121,0.072597
visitNumber,0.010963,1.000000,0.010963,-0.044609,NaN,0.033862,NaN,0.038065,0.046720,0.056410,0.352790,0.355427,0.036941,-0.024942
visitStartTime,1.000000,0.010963,1.000000,0.126059,NaN,-0.024275,NaN,-0.027545,0.000363,-0.015798,-0.021421,-0.026284,-0.025121,0.072597
device_isMobile,0.126059,-0.044609,0.126059,1.000000,NaN,-0.056735,NaN,-0.061053,-0.129534,-0.040219,-0.038764,-0.052527,-0.021927,-0.001714
totals_bounces,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals_hits,-0.024275,0.033862,-0.024275,-0.056735,NaN,1.000000,NaN,0.979798,0.664017,0.570926,0.147143,0.138177,0.330191,-0.029107
totals_newVisits,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
totals_pageviews,-0.027545,0.038065,-0.027545,-0.061053,NaN,0.979798,NaN,1.000000,0.681559,0.601160,0.144332,0.125006,0.374284,-0.031257
totals_sessionQualityDim,0.000363,0.046720,0.000363,-0.129534,NaN,0.664017,NaN,0.681559,1.000000,0.438278,0.066650,0.087668,0.073071,-0.020710
totals_timeOnSite,-0.015798,0.056410,-0.015798,-0.040219,NaN,0.570926,NaN,0.601160,0.438278,1.000000,0.107282,0.101257,0.219058,-0.024295


In [17]:
#add column for month

trainnew['month'] = trainnew['date'].apply(lambda s:s[4:6])
testnew['month'] = testnew['date'].apply(lambda s:s[4:6])
trainnew['month'].describe()

count     1708337
unique         12
top            11
freq       207661
Name: month, dtype: object

In [18]:
#remove target variables
testnew.drop(['totals_totalTransactionRevenue','totals_transactionRevenue' ], axis = 1, inplace = True)
trainnew.drop(['totals_totalTransactionRevenue','totals_transactionRevenue' ], axis = 1, inplace = True)


In [19]:
#histogram of dates

#trainnew['month'].hist(bins=14)

#plt.hist(trainnew['month'], bins=12)
#plt.ylabel('No of times')
#plt.show()



In [20]:
#lets fill in missing values to do random forest
#remove additional columns that are categorical but lots of vars, so would be hard to compute


trainnew.drop(['trafficSource_adwordsClickInfo.gclId','geoNetwork_networkDomain' ], axis = 1, inplace = True)
testnew.drop(['trafficSource_adwordsClickInfo.gclId','geoNetwork_networkDomain' ], axis = 1, inplace = True)

trainnew.shape


(1708337, 35)

In [21]:
naobjects = ['trafficSource_adContent', 'trafficSource_adwordsClickInfo.adNetworkType','trafficSource_adwordsClickInfo.isVideoAd',
'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign','trafficSource_isTrueDirect', 'trafficSource_keyword',
'trafficSource_medium', 'trafficSource_referralPath', 'trafficSource_source']

for x in naobjects:
    trainnew[x] = trainnew[x].fillna('None')
    testnew[x] = testnew[x].fillna('None')

zero = ['totals_bounces','trafficSource_adwordsClickInfo.page','totals_newVisits', 'totals_pageviews', 'totals_sessionQualityDim'
       ,'totals_timeOnSite', 'totals_transactions']    
for x in zero:
    trainnew[x] = trainnew[x].fillna(0)
    testnew[x] = testnew[x].fillna(0)

In [22]:
#drop additional columns that not useful or categorical with lots of unique values.

trainnew.drop(['fullVisitorId',  'visitId', 'visitStartTime', 'geoNetwork_country', 'geoNetwork_city', 'trafficSource_referralPath', 'trafficSource_keyword'], axis = 1, inplace = True)
testnew.drop(['fullVisitorId',  'visitId', 'visitStartTime', 'geoNetwork_country', 'geoNetwork_city', 'trafficSource_referralPath', 'trafficSource_keyword'], axis = 1, inplace = True)

In [23]:
#concat train/test, get dummies for concat df, then split back apart

concat = pd.concat((trainnew,testnew))
concat = pd.get_dummies(concat)
trainexpanded = concat[:len(train)]
testexpanded = concat[len(train):]

In [24]:
trainexpanded.shape

(1708337, 2258)

In [25]:
trainnew.columns

Index(['channelGrouping', 'date', 'visitNumber', 'device_browser',
       'device_deviceCategory', 'device_isMobile', 'device_operatingSystem',
       'geoNetwork_continent', 'geoNetwork_metro', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_sessionQualityDim',
       'totals_timeOnSite', 'totals_transactions', 'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
       'trafficSource_isTrueDirect', 'trafficSource_medium',
       'trafficSource_source', 'month'],
      dtype='object')

In [26]:
revenue = revenue.fillna(0)

In [35]:
#CV Setup
withid = trainexpanded
withid['fullVisitorId'] = train['fullVisitorId']

#test/train split
X_train, X_test, y_train, y_test = train_test_split(withid, revenue, test_size=0.2)
trainids = X_train['fullVisitorId']
testids = X_test['fullVisitorId']
X_train = X_train.drop('fullVisitorId', axis = 1)
X_test = X_test.drop('fullVisitorId', axis = 1)
modelval = RandomForestRegressor()
#fit model
modelval.fit(X_train, y_train)
print('model fitted.')

#make predictions, compare against true value, compute RMSE
predictionsval = modelval.predict(X_test)
predictionsval = pd.DataFrame(predictionsval)
predictionsval['fullVisitorId'] = testids
predsval = predictionsval[0].groupby(predictionsval['fullVisitorId']).sum()
predsval = pd.DataFrame(predsval)
predsval = predsval.reset_index()
predsval.columns = ['fullVisitorId', 'PredictedLogRevenue']
predsval['true'] = y_test
predsval['PredictedLogRevenue'] = predsval['PredictedLogRevenue'].apply(np.log1p)
predsval['PredictedLogRevenue'] = predsval['PredictedLogRevenue'].fillna(0)
predsval['true'] = predsval['true'].apply(np.log1p)
predsval['true'] = predsval['true'].fillna(0)
predsval['true'] = predsval['true'] + 1
diff = predsval['true'] - predsval['PredictedLogRevenue']
diff = diff**2
diff = diff/len(predsval)
RMSE = sum(diff) ** .5



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


model fitted.


In [ ]:
#train final model and make preds


revenue = revenue.fillna(0)

model = RandomForestRegressor()
model.fit(trainexpanded, revenue)

print('model fitted.')
predictions = model.predict(testexpanded)

In [74]:
predictions[:100]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [75]:
predictions = pd.DataFrame(predictions)


In [76]:
ids = test["fullVisitorId"]
predictions['fullVisitorId'] = ids

In [77]:
preds = predictions[0].groupby(predictions['fullVisitorId']).sum()

In [78]:
preds = pd.DataFrame(preds)

In [79]:
preds = preds.reset_index()

In [80]:
preds.columns = ['fullVisitorId', 'PredictedLogRevenue']

In [81]:
preds['PredictedLogRevenue'] = preds['PredictedLogRevenue'].apply(np.log1p)

In [82]:
preds['PredictedLogRevenue'] = preds['PredictedLogRevenue'].fillna(0)

In [84]:
preds.to_csv('predictions5.csv', index = False)

In [87]:
#look at feature importance

feature_importances = pd.DataFrame(model.feature_importances_,
                                   index = trainexpanded.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
feature_importances

,importance
visitNumber,0.281011
totals_transactions,0.099560
geoNetwork_region_Virginia,0.054870
totals_hits,0.051001
totals_timeOnSite,0.046685
totals_pageviews,0.027833
date_20170823,0.025703
date_20170214,0.018813
date_20170901,0.015079
device_operatingSystem_Windows,0.013662
